In [1]:
import os
from dotenv import load_dotenv,find_dotenv
_=load_dotenv(find_dotenv())
groq_api_key = os.getenv("GROQ_API_KEY")

### Step1: Extract Data from file

In [2]:
from langchain_community.document_loaders import PyPDFLoader

In [3]:
%pwd

'/home/sayandip-saha/Desktop/CODING/Project/File'

In [5]:
cd ..

/home/sayandip-saha/Desktop/CODING/Project


In [6]:
loader = PyPDFLoader("Data/rag_technique.pdf")

In [7]:
data = loader.load()

In [8]:
data

[Document(metadata={'source': 'Data/rag_technique.pdf', 'page': 0}, page_content='RAG \n©Harshit AhluwaliaDifferent Types of\nTechniques\nStandard RAG Corrective RAG Speculative RAG\nSelf RAG Agentic RAG Fusion RAG'),
 Document(metadata={'source': 'Data/rag_technique.pdf', 'page': 1}, page_content='©Harshit AhluwaliaStandard RAG\nStandard RAG combines a retrieval model (like a search\nengine) with a generative model (like GPT). The retrieval\nmodel fetches relevant documents or pieces of information\nfrom a large database, and the generative model uses this\ninformation to generate coherent and contextually accurate\nresponses or content.\n'),
 Document(metadata={'source': 'Data/rag_technique.pdf', 'page': 2}, page_content='©Harshit AhluwaliaHow it works?\nStep 1: Query Input - A user query or input is provided to the\nretrieval component of the system.\nStep 2: Retrieval Process - The retriever searches a large\ncorpus or database for documents or text passages that are\nmost relevant

In [9]:
len(data)

14

In [10]:
entire_data = ""
for page in data:
    entire_data += page.page_content

In [11]:
entire_data

'RAG \n©Harshit AhluwaliaDifferent Types of\nTechniques\nStandard RAG Corrective RAG Speculative RAG\nSelf RAG Agentic RAG Fusion RAG©Harshit AhluwaliaStandard RAG\nStandard RAG combines a retrieval model (like a search\nengine) with a generative model (like GPT). The retrieval\nmodel fetches relevant documents or pieces of information\nfrom a large database, and the generative model uses this\ninformation to generate coherent and contextually accurate\nresponses or content.\n©Harshit AhluwaliaHow it works?\nStep 1: Query Input - A user query or input is provided to the\nretrieval component of the system.\nStep 2: Retrieval Process - The retriever searches a large\ncorpus or database for documents or text passages that are\nmost relevant to the query. This is often done using vector\nsearch or dense retrieval methods, where both the query and\ndocuments are encoded into high-dimensional vectors.\nStep 3: Selection of Top Documents - The retriever ranks the\ndocuments based on their rel

### Step2: Get chunks out of data

In [12]:
from langchain.text_splitter import TokenTextSplitter

In [13]:
spliter = TokenTextSplitter(
    model_name = "gpt-3.5-turbo",
    chunk_size=10000,
    chunk_overlap=200,
)

In [14]:
from langchain.docstore.document import Document

In [15]:
chunk_question_gen = spliter.split_text(entire_data)

In [16]:
type(chunk_question_gen)

list

In [17]:
len(chunk_question_gen)

1

In [18]:
type(chunk_question_gen[0])

str

In [19]:
document_ques_gen = [Document(page_content=i) for i in chunk_question_gen]

In [20]:
type(document_ques_gen[0])

langchain_core.documents.base.Document

In [21]:
len(document_ques_gen)

1

In [22]:
document_ans_gen = spliter.split_documents(document_ques_gen) # split operation on document (list of Document datatype)

In [23]:
len(document_ans_gen)

1

In [24]:
document_ans_gen

[Document(page_content='RAG \n©Harshit AhluwaliaDifferent Types of\nTechniques\nStandard RAG Corrective RAG Speculative RAG\nSelf RAG Agentic RAG Fusion RAG©Harshit AhluwaliaStandard RAG\nStandard RAG combines a retrieval model (like a search\nengine) with a generative model (like GPT). The retrieval\nmodel fetches relevant documents or pieces of information\nfrom a large database, and the generative model uses this\ninformation to generate coherent and contextually accurate\nresponses or content.\n©Harshit AhluwaliaHow it works?\nStep 1: Query Input - A user query or input is provided to the\nretrieval component of the system.\nStep 2: Retrieval Process - The retriever searches a large\ncorpus or database for documents or text passages that are\nmost relevant to the query. This is often done using vector\nsearch or dense retrieval methods, where both the query and\ndocuments are encoded into high-dimensional vectors.\nStep 3: Selection of Top Documents - The retriever ranks the\ndocum

### Step3: Load LLM Model

In [25]:
from langchain_groq import ChatGroq

model = ChatGroq(
    model="openai/gpt-oss-20b",
    temperature=1.7, # Criteria for creativity
    max_tokens=4096 # max new tokens
)

### Step4: Define prompt template and generate questions

In [26]:
prompt_template = """
    You are an expert at creating questions based on moden GEN AI materials and Documentation.
    Your goal is to prepare user for their interviews regarding AI Engineering roles.
    You do this by asking questions about the document below
    
    {text}

    Create uestion out of the document provided to you. Make sure you don't loose any information regarding the document.

    Questions: 
"""

In [27]:
from langchain_core.prompts import PromptTemplate

In [28]:
Prompt_questions = PromptTemplate(template=prompt_template,input_variables=['text'])

In [29]:
refine_template = ("""
    You are an expert at creating questions based on moden GEN AI materials and Documentation.
    Your goal is to prepare user for their interviews regarding AI Engineering roles.
    we have recieved some practice questions to each certain extent : {existing_answer}.
    we have the option to refine the existing question orr add new ones.
    (only if necessary) with some more context below.

    {text}

    Given the new context, refine the original question in english. If the context is not useful please provide the original question.
    Question: 
""")

In [30]:
refine_prompt_question = PromptTemplate(
    input_variables=["existing_answer","text"],
    template=refine_template,
)

In [31]:
from langchain.chains.summarize import load_summarize_chain

In [32]:
Question_generation_chain = load_summarize_chain(
    llm=model,
    chain_type="refine",
    verbose=True,
    question_prompt=Prompt_questions,
    refine_prompt = refine_prompt_question
)

In [33]:
Ques = Question_generation_chain.run(document_ques_gen)

/tmp/ipykernel_4900/3100231773.py:1: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  Ques = Question_generation_chain.run(document_ques_gen)




> Entering new RefineDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

    You are an expert at creating questions based on moden GEN AI materials and Documentation.
    Your goal is to prepare user for their interviews regarding AI Engineering roles.
    You do this by asking questions about the document below

    RAG 
©Harshit AhluwaliaDifferent Types of
Techniques
Standard RAG Corrective RAG Speculative RAG
Self RAG Agentic RAG Fusion RAG©Harshit AhluwaliaStandard RAG
Standard RAG combines a retrieval model (like a search
engine) with a generative model (like GPT). The retrieval
model fetches relevant documents or pieces of information
from a large database, and the generative model uses this
information to generate coherent and contextually accurate
responses or content.
©Harshit AhluwaliaHow it works?
Step 1: Query Input - A user query or input is provided to the
retrieval component of the system.
Step 2: Retrieval Process - The retriever sear

In [34]:
print(Ques)

**Questions – RAG (Retrieval‑Augmented Generation) Primer**

1. **What is Standard RAG and what are the two core components it combines?**  
   a) Explain how the retrieval model and generative model cooperate.  
   b) Provide an example of a use‑case where this architecture shines.

2. **Walk through each step of the Standard RAG workflow (from query input to final output).**  
   – For every step, name the typical methods used (e.g., vector search, dense retrieval).

3. **Explain the objective of a Corrective RAG system and the main difference between it and Standard RAG.**  
   – Describe the validation step: what kinds of sources could serve as “trusted datasets”?  
   – How does the correction mechanism refine or regenerate a response?

4. **Speculative RAG attempts to handle ambiguity or multiple interpretations.**  
   – List the steps taken from retrieval to the final selection of a response.  
   – How is the feedback‑based ranking of the speculative answers usually implemente

### Step5 : Feed the questions and get the corresponding answer

In [35]:
from langchain_community.embeddings import HuggingFaceEmbeddings
# from langchain_community.vectorstores import FAISS
embedding_model = HuggingFaceEmbeddings(
    model_name = "sentence-transformers/all-MiniLM-L6-v2",
)

/tmp/ipykernel_4900/2795403779.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  embedding_model = HuggingFaceEmbeddings(
/home/sayandip-saha/Desktop/CODING/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-08-25 15:03:01.594972: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-08-25 15:03:02.766951: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not b

In [36]:
from langchain.vectorstores import FAISS

In [37]:
vector_store = FAISS.from_documents(document_ques_gen,embedding_model)

In [38]:
ques_list = Ques.split('\n')

In [39]:
print(ques_list)

['**Questions – RAG (Retrieval‑Augmented Generation) Primer**', '', '1. **What is Standard RAG and what are the two core components it combines?**  ', '   a) Explain how the retrieval model and generative model cooperate.  ', '   b) Provide an example of a use‑case where this architecture shines.', '', '2. **Walk through each step of the Standard RAG workflow (from query input to final output).**  ', '   – For every step, name the typical methods used (e.g., vector search, dense retrieval).', '', '3. **Explain the objective of a Corrective RAG system and the main difference between it and Standard RAG.**  ', '   – Describe the validation step: what kinds of sources could serve as “trusted datasets”?  ', '   – How does the correction mechanism refine or regenerate a response?', '', '4. **Speculative RAG attempts to handle ambiguity or multiple interpretations.**  ', '   – List the steps taken from retrieval to the final selection of a response.  ', '   – How is the feedback‑based rankin

In [40]:
Ques

'**Questions – RAG (Retrieval‑Augmented Generation) Primer**\n\n1. **What is Standard RAG and what are the two core components it combines?**  \n   a) Explain how the retrieval model and generative model cooperate.  \n   b) Provide an example of a use‑case where this architecture shines.\n\n2. **Walk through each step of the Standard RAG workflow (from query input to final output).**  \n   – For every step, name the typical methods used (e.g., vector search, dense retrieval).\n\n3. **Explain the objective of a Corrective RAG system and the main difference between it and Standard RAG.**  \n   – Describe the validation step: what kinds of sources could serve as “trusted datasets”?  \n   – How does the correction mechanism refine or regenerate a response?\n\n4. **Speculative RAG attempts to handle ambiguity or multiple interpretations.**  \n   – List the steps taken from retrieval to the final selection of a response.  \n   – How is the feedback‑based ranking of the speculative answers us

### Generating Answers

In [41]:
from langchain.chains import RetrievalQA

In [42]:
answer_generation_chain = RetrievalQA.from_chain_type(
    llm=model,
    chain_type="stuff",
    retriever = vector_store.as_retriever()
)

In [43]:
# Answer each question and save to a file
for question in ques_list:
    print("Question: ", question)
    answer = answer_generation_chain.run(question)
    print("Answer: ", answer)
    print("--------------------------------------------------\\n\\n")
    # Save answer to file
    with open("answers.txt", "a") as f:
        f.write("Question: " + question + "\\n")
        f.write("Answer: " + answer + "\\n")
        f.write("--------------------------------------------------\\n\\n")

Question:  **Questions – RAG (Retrieval‑Augmented Generation) Primer**
Answer:  **Retrieval‑Augmented Generation (RAG) Primer – Key Concepts, Types & How They Work**

---

### 1. What is RAG?

RAG blends a **retrieval model** (e.g., a search engine) with a **generative model** (e.g., GPT‑style language model).  
* Retrieval fetches relevant passages from a large database.  
* The generative model incorporates this retrieved data to produce coherent, context‑aware content that addresses a user’s query.

The core flow usually follows these stages:

1. **Query Input** – user statement or question.  
2. **Retrieval Process** – encode the query, perform vector search / dense retrieval across a corpus, pull back the most relevant documents.  
3. **Select Top‑K** – rank documents by relevance (top 3–5 commonly used).  
4. **Generate** – feed selected snippets to the generative model, which writes the final answer.  
5. **Output** – present the synthesized answer to the user.

---

### 2. Vari